In [1]:
# Import packages and read in datasets.
!pip install scProject -U
!pip install memory_profiler
import scanpy as sc
import scProject
import numpy as np
import sklearn

dataset = sc.read('4MCells.h5ad')
dataset.X = dataset.X * 100

patterns= sc.read('retina_patterns.h5ad')

scProject.matcher.orthologMapper(dataset, 'gene_map.csv', 'Gene stable ID','Mouse gene stable ID', 'gene_id')
dataset_filtered, patterns_filtered = scProject.matcher.filterAnnDatas(dataset, patterns, 'gene_id')






Requirement already up-to-date: scProject in /home/abaraban/anaconda3/envs/scProjections/lib/python3.7/site-packages (1.0.9.8996)
(4062980, 2403) dataset filter shape
(80, 2403) patterns filter shape


Trying to set attribute `.X` of view, copying.
Trying to set attribute `.X` of view, copying.


In [2]:
# Perfrom regression with low regularization and 50-50 Lasso Ridge
dataset = None
import matplotlib.pyplot as plt
import timeit
dataset_filtered = scProject.matcher.logTransform(dataset_filtered)
plt.rcParams['figure.figsize']= [10, 12]
startTime = timeit.default_timer()
%load_ext memory_profiler

%memit scProject.rg.NNLR_ElasticNet(dataset_filtered, patterns_filtered, '4M75', .000001, .5, layer='log')
%time scProject.rg.NNLR_ElasticNet(dataset_filtered, patterns_filtered, '4M75', .000001, .5, layer='log')
endTime = timeit.default_timer()
print(f"Regression completed in {endTime - startTime:0.4f} seconds")
scProject.viz.pearsonMatrix(dataset_filtered, patterns_filtered, 'Main_cluster_name', 77, '4M75', '4M75Pears', plot=True, display=False, path='/home/abaraban/conda_env/plots/pearsmat.pdf', xtickSize=6, ytickSize=5)

A layer named 'log' has been added to your filtered dataset
Regressing on loglayer of dataset_filtered
(4062980, 80)
peak memory: 122333.39 MiB, increment: 45866.24 MiB
Regressing on loglayer of dataset_filtered
(4062980, 80)
CPU times: user 28min 22s, sys: 3min 16s, total: 31min 38s
Wall time: 14min 18s
Regression completed in 1718.1572 seconds
['Retinal progenitors and Muller glia', 'Amacrine cells', 'Photoreceptor cells', 'Bipolar cells', 'Ganglion cells', ..., 'PDE1C_ACSM3 positive cells', 'MUC13_DMBT1 positive cells', 'Thymocytes', 'Antigen presenting cells', 'Thymic epithelial cells']
Length: 77
Categories (77, object): ['Retinal progenitors and Muller glia', 'Amacrine cells', 'Photoreceptor cells', 'Bipolar cells', ..., 'MUC13_DMBT1 positive cells', 'Thymocytes', 'Antigen presenting cells', 'Thymic epithelial cells']


In [ ]:
# Calculate and store the UMAP coordinates

scProject.viz.UMAP_Projection(dataset_filtered, 'Main_cluster_name', '4M75', '4M75UMAP', 25, plot=False)






In [ ]:
# Visualize UMAP for a subset of cells and save in folder called plots

plt.rcParams['figure.figsize']= [12, 10]
subset=["Acinar cells", "Photoreceptor cells", "Microglia", "Retinal progenitors and Muller glia", "Goblet cells",
        "Myeloid cells","Astrocytes", "Vascular endothelial cells", "Oligodendrocytes", "Inhibitory interneurons", "Excitatory neurons"]
scProject.viz.UMAP_Viz(dataset_filtered, '4M75UMAP', 'Main_cluster_name', pointSize=.0001,
                       subset=subset, path="/home/abaraban/conda_env/plots/UMAP75.png", display=False)


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt  


outputs = dataset_filtered.obs['Main_cluster_name']
classes = outputs.unique()
Predscores = np.zeros((80,len(classes)))
AUCScores = np.zeros((80,len(classes)))
meanPatternWeightInCellType = np.zeros((80,len(classes)))
targetCellType = []
curves = {}
for i in range(80):
    model = RandomForestClassifier(class_weight = 'balanced', n_jobs=-1) 
    pattern_weights = dataset_filtered.obsm['4M75'][:,i]
    X_train, X_test, y_train, y_test = train_test_split(dataset_filtered.obsm['4M75'][:,i].reshape(-1,1), dataset_filtered.obs['Main_cluster_name'], test_size = 0.3, random_state=613, stratify=dataset_filtered.obs['Main_cluster_name'] )
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    # print(predictions)
    for j in range(len(classes)):
        class_at_hand = model.classes_[j]
        indices = [] # index of all occurences of a class
        for idx in range(len(y_test)):
            if y_test[idx] == class_at_hand:
                indices.append(idx)
        # print(len(indices))
        score = 0
        for predict, truth in zip(predictions[indices], y_test[indices]):
            if predict == truth:
                score +=1
        score = score/len(indices)
        # prediction score
        Predscores[i,j] = score
        
        #ROC AUC score
        binarize = {class_at_hand: 1}
        biny_test = [binarize.get(x, 0) for x in y_test]
        # bi_predictions = [binarize.get(x,0) for x in predictions]
        bi_predictions = model.predict_proba(X_test)[:, j]
        score = roc_auc_score(biny_test, bi_predictions)
        AUCScores[i,j] = score
        
        #Mean Pattern Weight
        meanPatternWeightInCellType[i,j] = np.mean(X_test[indices])
        
        print(i, class_at_hand)
        print(AUCScores[i,j], Predscores[i,j])
        # print(meanPatternWeightInCellType[i,j])

0 AFP_ALB positive cells
0.5011650431271071 0.0
0 Acinar cells
0.5011682646662465 0.0
0 Adrenocortical cells
0.5012676104115449 0.0
0 Amacrine cells
0.500937977371457 0.0
0 Antigen presenting cells
0.5011651243811303 0.0
0 Astrocytes
0.5011999247002834 0.0
0 Bipolar cells
0.5011653366509866 0.0
0 Bronchiolar and alveolar epithelial cells
0.5011922400903982 0.0
0 CCL19_CCL21 positive cells
0.5011650498181865 0.0
0 CLC_IL5RA positive cells
0.5011650201868463 0.0
0 CSH1_CSH2 positive cells
0.5011650220985002 1.0
0 Cardiomyocytes
0.5010844719478766 0.0
0 Chromaffin cells
0.4977149901354165 0.00015832805573147562
0 Ciliated epithelial cells
0.501164997848982 0.0
0 Corneal and conjunctival epithelial cells
0.5011650201868463 0.0
0 Ductal cells
0.5011663771258661 0.0
0 ELF3_AGBL2 positive cells
0.5011650373919572 0.0
0 ENS glia
0.5011659145696625 0.0
0 ENS neurons
0.4991611558806075 0.0
0 Endocardial cells
0.5011671152937309 0.0
0 Epicardial fat cells
0.5011655327346435 0.0
0 Erythroblasts
0.

2 Ductal cells
0.3557907606008341 0.0013802622498274672
2 ELF3_AGBL2 positive cells
0.5100089320091512 0.0
2 ENS glia
0.49069614121341854 0.0
2 ENS neurons
0.5223814744042306 0.0
2 Endocardial cells
0.3358056723293014 0.005858494817485353
2 Epicardial fat cells
0.4474098736222124 0.003527336860670194
2 Erythroblasts
0.5389642713269859 0.000642306594347702
2 Excitatory neurons
0.4941921060841349 0.01738405115915741
2 Extravillous trophoblasts
0.2027773465008476 0.006507592190889371
2 Ganglion cells
0.5158385856251169 0.00032862306933946765
2 Goblet cells
0.516567355719198 0.0009199632014719411
2 Granule neurons
0.5198355541081926 0.002814408920823428
2 Hematopoietic stem cells
0.533638760933824 0.0
2 Hepatoblasts
0.5071440168890299 0.0019961218204631002
2 Horizontal cells
0.5355629020797973 0.0
2 IGFBP1_DKK1 positive cells
0.34411220081114824 0.0
2 Inhibitory interneurons
0.5111100268454528 0.0024379603254035464
2 Inhibitory neurons
0.5163877258121555 0.005207739384954575
2 Intestinal e

4 Chromaffin cells
0.5061386018594661 0.0
4 Ciliated epithelial cells
0.4849382939664159 0.0
4 Corneal and conjunctival epithelial cells
0.3451189425533627 0.0
4 Ductal cells
0.46309232375382675 0.0006901311249137336
4 ELF3_AGBL2 positive cells
0.5064335497950929 0.0
4 ENS glia
0.4949388205428292 0.0
4 ENS neurons
0.5057737333288795 0.0
4 Endocardial cells
0.4959045814408989 0.0004506534474988734
4 Epicardial fat cells
0.48783725247515347 0.0
4 Erythroblasts
0.4983453106057333 0.000642306594347702
4 Excitatory neurons
0.5076910290029527 2.1183916111692196e-05
4 Extravillous trophoblasts
0.46831336632367 0.0
4 Ganglion cells
0.5064526821131865 0.0
4 Goblet cells
0.4572507213826409 0.0009199632014719411
4 Granule neurons
0.5067022136550002 0.0
4 Hematopoietic stem cells
0.41347729541413236 0.0
4 Hepatoblasts
0.4994246985037427 0.0003421923120793886
4 Horizontal cells
0.5057168121909473 0.0
4 IGFBP1_DKK1 positive cells
0.494511321814585 0.0
4 Inhibitory interneurons
0.5066180874962893 0.0

6 Corneal and conjunctival epithelial cells
0.5895125210954799 0.0
6 Ductal cells
0.5833115422026411 0.0
6 ELF3_AGBL2 positive cells
0.5793068898198286 0.0
6 ENS glia
0.5828095650906444 0.0
6 ENS neurons
0.5367806313090545 0.0
6 Endocardial cells
0.5883059231578969 0.0
6 Epicardial fat cells
0.5824669976461737 0.0
6 Erythroblasts
0.5897371454480675 0.0
6 Excitatory neurons
0.5391617225544905 0.10184167670696025
6 Extravillous trophoblasts
0.5895449318920285 0.0
6 Ganglion cells
0.46992832363087217 0.0013144922773578706
6 Goblet cells
0.5826409351330943 0.0
6 Granule neurons
0.515627119012032 0.00925343539119218
6 Hematopoietic stem cells
0.5880052246914524 0.0
6 Hepatoblasts
0.5891009959348836 0.0
6 Horizontal cells
0.47016722384233123 0.0
6 IGFBP1_DKK1 positive cells
0.5895199247713107 0.0
6 Inhibitory interneurons
0.469067808757936 0.015885236225524162
6 Inhibitory neurons
0.5345758035133781 0.07146386893222337
6 Intestinal epithelial cells
0.5869752739535182 0.0
6 Islet endocrine ce

8 Epicardial fat cells
0.2007169779379813 0.0
8 Erythroblasts
0.5085818459317842 0.0021410219811590064
8 Excitatory neurons
0.5641315132072067 0.0008976684452329569
8 Extravillous trophoblasts
0.15456437768411885 0.0021691973969631237
8 Ganglion cells
0.5600952472093392 0.0
8 Goblet cells
0.5021982204730872 0.0
8 Granule neurons
0.5628175651684018 8.528511881283115e-05
8 Hematopoietic stem cells
0.4434721656821492 0.0
8 Hepatoblasts
0.5181386532746254 0.0017679936124101745
8 Horizontal cells
0.5594576684736033 0.0
8 IGFBP1_DKK1 positive cells
0.1749272341546877 0.0
8 Inhibitory interneurons
0.563353889294925 2.5662740267405754e-05
8 Inhibitory neurons
0.5664248918800394 1.9006348120272173e-05
8 Intestinal epithelial cells
0.48831831639006584 0.0019011406844106464
8 Islet endocrine cells
0.5119384098960907 0.0
8 Lens fibre cells
0.5367683053658013 0.0
8 Limbic system neurons
0.5638997907919056 0.0
8 Lymphatic endothelial cells
0.25492901475272023 0.020072992700729927
8 Lymphoid cells
0.

...

71 Thymic epithelial cells
0.5079417237712665 0.0
71 Thymocytes
0.5020983706760781 0.0
71 Trophoblast giant cells
0.5049969668245502 0.0
71 Unipolar brush cells
0.5052057868698425 0.00012663036596175764
71 Ureteric bud cells
0.48664265248195665 0.0007296607077708865
71 Vascular endothelial cells
0.4989301665755249 0.0008876658825618037
71 Visceral neurons
0.4942588763933342 0.0
72 AFP_ALB positive cells
0.5447614492151958 1.0
72 Acinar cells
0.5309391009509572 0.0
72 Adrenocortical cells
0.5452689058618886 4.053670598727148e-05
72 Amacrine cells
0.49251754914002416 0.0
72 Antigen presenting cells
0.5197528172215229 0.0
72 Astrocytes
0.501885776088935 0.006276720596864784
72 Bipolar cells
0.5101609131233774 0.0
72 Bronchiolar and alveolar epithelial cells
0.5006601620953604 0.0031275720164609055
72 CCL19_CCL21 positive cells
0.47216255451543876 0.0
72 CLC_IL5RA positive cells
0.48023327028232793 0.0
72 CSH1_CSH2 positive cells
0.5144551496699117 0.0
72 Cardiomyocytes
0.50750142509745 0.

73 Thymic epithelial cells
0.49438647098958416 0.0
73 Thymocytes
0.49877337511776476 0.0
73 Trophoblast giant cells
0.4983913591971409 0.0
73 Unipolar brush cells
0.5039701050819426 6.331518298087882e-05
73 Ureteric bud cells
0.489505929822269 0.0
73 Vascular endothelial cells
0.4996875663547916 0.0003550663530247215
73 Visceral neurons
0.4880669876265801 0.0
74 AFP_ALB positive cells
0.284358468107019 0.0
74 Acinar cells
0.4945060418410077 0.000585480093676815
74 Adrenocortical cells
0.5156198029062972 0.03406096720580486
74 Amacrine cells
0.5174892092053129 0.0
74 Antigen presenting cells
0.40478369712017354 0.0
74 Astrocytes
0.5129256236376416 0.0009535585547824629
74 Bipolar cells
0.527116599934496 0.0
74 Bronchiolar and alveolar epithelial cells
0.5039195773186853 0.001382716049382716
74 CCL19_CCL21 positive cells
0.3765811740123914 0.0
74 CLC_IL5RA positive cells
0.4976316505231621 0.0
74 CSH1_CSH2 positive cells
0.5147232340337214 0.0
74 Cardiomyocytes
0.5092043901948158 0.00059

75 Ureteric bud cells
0.5147117605539305 0.0
75 Vascular endothelial cells
0.5139239049839893 4.438329412809019e-05
75 Visceral neurons
0.4948317718452264 0.0
76 AFP_ALB positive cells
0.4989151076631868 0.0
76 Acinar cells
0.46525515332191486 0.00117096018735363
76 Adrenocortical cells
0.49962364265338605 0.002158579593822206
76 Amacrine cells
0.5047956808173422 0.0
76 Antigen presenting cells
0.47940187859310174 0.0
76 Astrocytes
0.5021747325994285 0.000534411937295666
76 Bipolar cells
0.5038660050197526 0.0
76 Bronchiolar and alveolar epithelial cells
0.4950335665145692 0.0018436213991769547
76 CCL19_CCL21 positive cells
0.49993829364718084 0.0
76 CLC_IL5RA positive cells
0.4918801110066804 0.0
76 CSH1_CSH2 positive cells
0.5080037838605058 0.0
76 Cardiomyocytes
0.4935111827428096 0.0033032588867524526
76 Chromaffin cells
0.48970939432451766 0.00047498416719442683
76 Ciliated epithelial cells
0.4388639251940123 0.0
76 Corneal and conjunctival epithelial cells
0.5079931050495421 0.0


77 Ureteric bud cells
0.4491863591157839 0.003283473184968989
77 Vascular endothelial cells
0.5009613222815902 0.017708934357107985
77 Visceral neurons
0.5003282559854862 0.0
78 AFP_ALB positive cells
0.49940981838976567 0.0
78 Acinar cells
0.5052002471513477 0.0
78 Adrenocortical cells
0.5023347550743669 0.0021889821233126595
78 Amacrine cells
0.5052904336823738 0.0
78 Antigen presenting cells
0.501343362623982 0.0
78 Astrocytes
0.500023070958975 0.0018337664515047363
78 Bipolar cells
0.48353269860647285 0.0
78 Bronchiolar and alveolar epithelial cells
0.4994005067350186 0.0011522633744855968
78 CCL19_CCL21 positive cells
0.49236724307607876 0.0
78 CLC_IL5RA positive cells
0.46011192662818257 0.0
78 CSH1_CSH2 positive cells
0.4933490439770483 0.0
78 Cardiomyocytes
0.4936410100257286 0.0020213972292067246
78 Chromaffin cells
0.49050534574569954 0.001266624445851805
78 Ciliated epithelial cells
0.5050261911634534 0.0
78 Corneal and conjunctival epithelial cells
0.49237396099955977 0.0
7

79 Unipolar brush cells
0.5119050560228591 0.0
79 Ureteric bud cells
0.41578029096181945 0.0036483035388544327
79 Vascular endothelial cells
0.5015533731134683 0.0008876658825618037
79 Visceral neurons
0.5043966703277506 0.0


In [11]:
import pandas as pd
dfMeanPatternWeight = pd.DataFrame(meanPatternWeightInCellType, columns=model.classes_, index=["Feature " + str(x+1) for x in range(80)])
dfPredScores = pd.DataFrame(Predscores, columns=model.classes_, index=["Feature " + str(x+1) for x in range(80)])
dfAUCScores = pd.DataFrame(AUCScores, columns=model.classes_, index=["Feature " + str(x+1) for x in range(80)])
dfMeanPatternWeight.to_csv('4millionMeanPatternWeight.csv')
dfPredScores.to_csv('4millionPredScores.csv')
dfAUCScores.to_csv('4millionAUCScores.csv')

In [ ]:
subset=["Acinar cells"]
scProject.viz.featurePlots(dataset_filtered, [23], '4M75', '4M75UMAP', obsColumn='Main_cluster_name', pointSize=.05, subset=subset,
                           path="/home/abaraban/conda_env/plots/acinar", display=False, clip=99)

In [ ]:
print(scProject.stats.importantGenes(patterns_filtered, 23, .01), "Feature 23 Genes")


<pre>
ENSMUSG00000026628	activating transcription factor 3	Atf3
ENSMUSG00000036478	BTG anti-proliferation factor 1 Btg1
ENSMUSG00000020423	BTG anti-proliferation factor 2	Btg2
ENSMUSG00000001175	calmodulin 1	Calm1
ENSMUSG00000028195	cellular communication network factor 1	Ccn1
ENSMUSG00000001270	creatine kinase, brain	Ckb	B-CK, Bck, CPK-B, Ck-3, Ck3	MGI:88407
ENSMUSG00000024190	dual specificity phosphatase 1	Dusp1	3CH134, MKP, MKP1, Ptp, Ptpn16	MGI:105120
ENSMUSG00000038418	early growth response 1	Egr1
ENSMUSG00000021250	FBJ osteosarcoma oncogene	Fos	
ENSMUSG00000003545	FBJ osteosarcoma oncogene B	Fosb
ENSMUSG00000036181	H1.2 linker histone, cluster member	H1f2
ENSMUSG00000022528	hes family bHLH transcription factor 1	Hes1
ENSMUSG00000048001	hes family bHLH transcription factor 5	Hes5
ENSMUSG00000042745	inhibitor of DNA binding 1, HLH protein	Id1
ENSMUSG00000053560	immediate early response 2	Ier2
ENSMUSG00000022108	integral membrane protein 2B	Itm2b
ENSMUSG00000052684	jun proto-oncogene	Jun
ENSMUSG00000052837	jun B proto-oncogene	Junb
ENSMUSG00000055148	Kruppel-like factor 2 (lung)	Klf2	
ENSMUSG00000049775	thymosin, beta 4, X chromosome	Tmsb4x
ENSMUSG00000045690	WD repeat domain 89	Wdr89
</pre>


In [ ]:
scProject.viz.patternWeightDistribution(dataset_filtered, '4M75', [22], obsColumn='Main_cluster_name', subset=['Acinar cells'], numBins=150)

In [ ]:
# Getting all of the Acinar cells and separating by expression of feature 23
acinar = dataset_filtered[dataset_filtered.obs['Main_cluster_name'].isin(['Acinar cells'])]
print(acinar.shape)
Acinar1 = acinar[acinar.obsm['4M75'][:, 22] > 0.05]
Acinar2 = acinar[acinar.obsm['4M75'][:, 22] <= 0.05]
Acinar1.X = np.log1p(Acinar1.X)
Acinar2.X = np.log1p(Acinar2.X)

# In order to do the regression we had to filter genes, but now we are going to use the whole feature map
AllGenes = dataset[dataset.obs['Main_cluster_name'].isin(['Acinar cells'])]
# transfer regression insights
AllGenes.obsm['4M75'] = acinar.obsm['4M75']

cluster1 = AllGenes[acinar.obsm['4M75'][:, 22] > .05]
cluster2 = AllGenes[acinar.obsm['4M75'][:, 22] <= .05]

cluster1.X = np.log1p(cluster1.X) #log transform for stats
cluster2.X = np.log1p(cluster2.X) #log transform for stats
print(cluster1.shape, cluster2.shape)

In [ ]:
# Here we confirm that the expression of feature 23 is significant with confidence .999999
scProject.stats.featureExpressionSig(Acinar1, '4M75', 23, .999999)

In [ ]:
# This tests whether the differential feature expression correlates with differential gene expression
# The p-value is not super informative as they trend towards 0
scProject.stats.HotellingT2(Acinar1, Acinar2)

In [ ]:

df = scProject.stats.BonferroniCorrectedDifferenceMeans(cluster1, cluster2, .9999999999, 'gene_short_name', verbose=613)
driver = scProject.stats.projectionDriver(patterns_filtered, Acinar1, Acinar2, .995, 'gene_short_name', 23)






In [ ]:
ductal = dataset_filtered[dataset_filtered.obs['Main_cluster_name'].isin(['Ductal cells'])]
print(ductal.shape)
d1 = ductal[ductal.obsm['4M75'][:, 22] > 0]
d2 = ductal[ductal.obsm['4M75'][:, 22] == 0]
d1.X = np.log1p(d1.X)
d2.X = np.log1p(d2.X)
print(d1.X.shape)
print(d2.X.shape)

In [ ]:
scProject.stats.HotellingT2(d1, d2)

In [ ]:
print(scProject.stats.BonferroniCorrectedDifferenceMeans(d1, d2, .9999999, 'gene_short_name'))

In [ ]:
print(ductal.obsm['4M75'][:, 22].shape)
np.histogram(ductal.obsm['4M75'][:, 22])

In [ ]:
import matplotlib.pyplot as plt
print(np.max(acinar.obsm['4M75'][:, 22]))
plt.hist(np.reshape(acinar.obsm['4M75'][:, 5], -1), bins=[0, .01, .02, .03, .04, .05, .6, .7, .8, .9 , 1, 1.1])
print(acinar.obsm['4M75'].shape)

In [ ]:
l=cluster1.X.todense()
l.todense()